In [1]:
# https://blog.csdn.net/qq_23869697/article/details/85106365

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential,load_model,Model
from tensorflow.keras.layers import Dense,LSTM,Flatten,Dropout,Activation,Input,Conv1D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split


# my_seed = 7
# tf.compat.v1.set_random_seed(my_seed)
# physical_devices = tf.config.list_physical_devices('GPU')
# try:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)
#     assert tf.config.experimental.get_memory_growth(physical_devices[0])
# except:
#     pass

In [3]:
help(Conv1D)

Help on class Conv1D in module keras.layers.convolutional:

class Conv1D(Conv)
 |  Conv1D(*args, **kwargs)
 |  
 |  1D convolution layer (e.g. temporal convolution).
 |  
 |  This layer creates a convolution kernel that is convolved
 |  with the layer input over a single spatial (or temporal) dimension
 |  to produce a tensor of outputs.
 |  If `use_bias` is True, a bias vector is created and added to the outputs.
 |  Finally, if `activation` is not `None`,
 |  it is applied to the outputs as well.
 |  
 |  When using this layer as the first layer in a model,
 |  provide an `input_shape` argument
 |  (tuple of integers or `None`, e.g.
 |  `(10, 128)` for sequences of 10 vectors of 128-dimensional vectors,
 |  or `(None, 128)` for variable-length sequences of 128-dimensional vectors.
 |  
 |  Examples:
 |  
 |  >>> # The inputs are 128-length vectors with 10 timesteps, and the batch size
 |  >>> # is 4.
 |  >>> input_shape = (4, 10, 128)
 |  >>> x = tf.random.normal(input_shape)
 |  >>>

In [4]:
data = pd.read_csv('data/da1.csv',index_col='time')
data = data.iloc[:,[0,1,2,5,6,7]]
data

,number,weekday,isRest,severity_num,priority_num,category
time,,,,,,
2002/9/23,206,2,0,18,13,3
2002/9/24,192,3,0,29,18,3
2002/9/25,245,4,0,34,16,3
2002/9/26,209,5,0,18,20,3
2002/9/27,225,6,0,30,27,3
...,...,...,...,...,...,...
2004/5/10,108,2,0,11,6,2
2004/5/11,110,3,0,18,8,2
2004/5/12,114,4,0,15,8,2


In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(data.iloc[:,[0]].values)
# dataset = scaler.fit_transform(data.iloc[:,[0,1,2,3,4]].values)

In [6]:
dataset

array([[0.67672414],
       [0.61637931],
       [0.84482759],
       [0.68965517],
       [0.75862069],
       [0.45258621],
       [0.43103448],
       [0.75      ],
       [0.72844828],
       [0.70689655],
       [0.84051724],
       [0.64655172],
       [0.25      ],
       [0.47844828],
       [0.68965517],
       [0.71982759],
       [0.82327586],
       [0.76724138],
       [0.60344828],
       [0.24137931],
       [0.2887931 ],
       [0.65086207],
       [0.63362069],
       [0.71982759],
       [1.        ],
       [0.79741379],
       [0.44396552],
       [0.34913793],
       [0.7887931 ],
       [0.77586207],
       [0.86206897],
       [0.75431034],
       [0.64224138],
       [0.31465517],
       [0.21982759],
       [0.52586207],
       [0.61206897],
       [0.73706897],
       [0.68534483],
       [0.48706897],
       [0.27155172],
       [0.2887931 ],
       [0.41810345],
       [0.67672414],
       [0.68965517],
       [0.72844828],
       [0.38793103],
       [0.495

In [7]:
def create_dataset(data,dataset,lookback):
    dataX,dataY = [],[]
    for i in range(len(data) - lookback):
        a = dataset[i:i+lookback,[0,1,2,3,4]]
#         a = dataset[i:i+lookback,[0,1,2]]
#         print(a)
        dataX.append(a)
        dataY.append(data.iloc[i+lookback,[5]])
    return np.array(dataX),np.array(dataY)

In [8]:
def create_dataset(data,dataset,lookback):
    dataX,dataY = [],[]
    for i in range(len(data) - lookback):
#         a = dataset[i:i+lookback,[0,1,2,3,4]]
        a = dataset[i:i+lookback,[0]]
#         print(a)
        dataX.append(a)
        dataY.append(data.iloc[i+lookback,[5]])
    return np.array(dataX),np.array(dataY)

In [9]:
lookback = 7
dataX,dataY = create_dataset(data,dataset,lookback)

In [10]:
dataX.shape,dataY.shape

((593, 7, 1), (593, 1))

In [11]:
from tensorflow.keras.utils import to_categorical

In [12]:
# 将category列进行one-hot编码
one_hots = to_categorical(dataY-1)

In [13]:
one_hots.shape

(593, 3)

In [14]:
# trainX,testX,trainY,testY = train_test_split(dataX,one_hots,test_size=0.20, random_state=1)
trainX,testX,trainY,testY = train_test_split(dataX,one_hots,test_size=0.20)

In [15]:
trainX.shape,testX.shape,trainY.shape,testY.shape

((474, 7, 1), (119, 7, 1), (474, 3), (119, 3))

In [16]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [17]:
inputs = Input(shape=(trainX.shape[1],trainX.shape[2]))
y = Conv1D(16, 3, activation='relu')(inputs)
z = Conv1D(16, 3, activation='relu')(y)
t = Conv1D(16, 3, activation='relu')(z)
o = Flatten()(t)
# o = Dense(32, activation='relu')(o)
outputs = Dense(3, activation = 'softmax')(o)

In [18]:
model = Model(inputs = inputs,outputs = outputs)

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [20]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 7, 1)]            0         
                                                                 
 conv1d (Conv1D)             (None, 5, 16)             64        
                                                                 
 conv1d_1 (Conv1D)           (None, 3, 16)             784       
                                                                 
 conv1d_2 (Conv1D)           (None, 1, 16)             784       
                                                                 
 flatten (Flatten)           (None, 16)                0         
                                                                 
 dense (Dense)               (None, 3)                 51        
                                                                 
Total params: 1,683
Trainable params: 1,683
Non-trainable par

In [21]:
history = model.fit(trainX,trainY,epochs=100, batch_size=128, verbose=2,validation_split=0.2,callbacks = [callback])
# history = model.fit(trainX,trainY,epochs=100, batch_size=12, verbose=2,validation_split=0.2)

Epoch 1/100
3/3 - 0s - loss: 1.1050 - accuracy: 0.3034 - val_loss: 1.1128 - val_accuracy: 0.3474 - 387ms/epoch - 129ms/step
Epoch 2/100
3/3 - 0s - loss: 1.1027 - accuracy: 0.3272 - val_loss: 1.1095 - val_accuracy: 0.3895 - 16ms/epoch - 5ms/step
Epoch 3/100
3/3 - 0s - loss: 1.1011 - accuracy: 0.3509 - val_loss: 1.1066 - val_accuracy: 0.3684 - 16ms/epoch - 5ms/step
Epoch 4/100
3/3 - 0s - loss: 1.0994 - accuracy: 0.3799 - val_loss: 1.1038 - val_accuracy: 0.3474 - 17ms/epoch - 6ms/step
Epoch 5/100
3/3 - 0s - loss: 1.0977 - accuracy: 0.3720 - val_loss: 1.1010 - val_accuracy: 0.3474 - 16ms/epoch - 5ms/step
Epoch 6/100
3/3 - 0s - loss: 1.0959 - accuracy: 0.3720 - val_loss: 1.0984 - val_accuracy: 0.3474 - 17ms/epoch - 6ms/step
Epoch 7/100
3/3 - 0s - loss: 1.0943 - accuracy: 0.3826 - val_loss: 1.0959 - val_accuracy: 0.3684 - 17ms/epoch - 6ms/step
Epoch 8/100
3/3 - 0s - loss: 1.0926 - accuracy: 0.4011 - val_loss: 1.0932 - val_accuracy: 0.3684 - 16ms/epoch - 5ms/step
Epoch 9/100
3/3 - 0s - loss: 

Epoch 69/100
3/3 - 0s - loss: 0.7648 - accuracy: 0.6227 - val_loss: 0.6651 - val_accuracy: 0.6947 - 16ms/epoch - 5ms/step
Epoch 70/100
3/3 - 0s - loss: 0.7608 - accuracy: 0.6385 - val_loss: 0.6681 - val_accuracy: 0.6947 - 17ms/epoch - 6ms/step
Epoch 71/100
3/3 - 0s - loss: 0.7577 - accuracy: 0.6438 - val_loss: 0.6646 - val_accuracy: 0.6842 - 16ms/epoch - 5ms/step
Epoch 72/100
3/3 - 0s - loss: 0.7552 - accuracy: 0.6385 - val_loss: 0.6597 - val_accuracy: 0.6947 - 16ms/epoch - 5ms/step
Epoch 73/100
3/3 - 0s - loss: 0.7543 - accuracy: 0.6385 - val_loss: 0.6612 - val_accuracy: 0.6842 - 16ms/epoch - 5ms/step
Epoch 74/100
3/3 - 0s - loss: 0.7503 - accuracy: 0.6438 - val_loss: 0.6561 - val_accuracy: 0.6947 - 16ms/epoch - 5ms/step
Epoch 75/100
3/3 - 0s - loss: 0.7473 - accuracy: 0.6464 - val_loss: 0.6561 - val_accuracy: 0.6947 - 16ms/epoch - 5ms/step
Epoch 76/100
3/3 - 0s - loss: 0.7472 - accuracy: 0.6491 - val_loss: 0.6538 - val_accuracy: 0.7053 - 15ms/epoch - 5ms/step
Epoch 77/100
3/3 - 0s - 

In [22]:
accuracy = model.evaluate(testX,testY, batch_size=128, verbose=2)

1/1 - 0s - loss: 0.6492 - accuracy: 0.7143 - 11ms/epoch - 11ms/step


In [23]:
testPredict = model.predict(testX)

In [24]:
# model = load_model('model/fcn11.h5')
# model = load_model('fcn11.h5')
# testPredict = model.predict(testX)

In [25]:
testPredict

array([[1.17242746e-02, 2.23146409e-01, 7.65129268e-01],
       [5.76221406e-01, 3.27096254e-01, 9.66823250e-02],
       [8.16673756e-01, 1.69626504e-01, 1.36997635e-02],
       [6.94589972e-01, 2.81336099e-01, 2.40739994e-02],
       [5.02105355e-01, 3.90928149e-01, 1.06966555e-01],
       [3.23836431e-02, 3.91554862e-01, 5.76061487e-01],
       [2.99441934e-01, 5.43703556e-01, 1.56854555e-01],
       [3.93816113e-01, 4.58943516e-01, 1.47240415e-01],
       [5.42961478e-01, 4.02028322e-01, 5.50102517e-02],
       [7.23983049e-01, 2.52425432e-01, 2.35914364e-02],
       [5.98884821e-01, 3.60729873e-01, 4.03853282e-02],
       [4.02089953e-03, 1.13204807e-01, 8.82774293e-01],
       [6.98535562e-01, 2.68074483e-01, 3.33899409e-02],
       [3.08503747e-01, 4.80565220e-01, 2.10931078e-01],
       [3.66360605e-01, 3.57404321e-01, 2.76235104e-01],
       [5.78432083e-01, 3.21350366e-01, 1.00217551e-01],
       [3.71914148e-01, 4.54255134e-01, 1.73830703e-01],
       [6.69248462e-01, 2.98399

In [26]:
testPredict.shape

(119, 3)

In [27]:
y_pred = np.argmax(testPredict, axis=1)

In [28]:
y_pred

array([2, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 2, 0, 1, 0, 0, 1, 0, 0, 0, 2, 1,
       0, 1, 0, 2, 2, 0, 2, 1, 0, 2, 2, 0, 0, 0, 2, 0, 2, 1, 0, 2, 0, 2,
       1, 2, 2, 1, 1, 1, 1, 2, 0, 2, 2, 1, 0, 0, 1, 0, 0, 0, 1, 2, 0, 2,
       2, 1, 2, 2, 1, 2, 1, 2, 1, 0, 0, 0, 2, 1, 1, 2, 0, 2, 1, 1, 0, 1,
       2, 2, 1, 1, 2, 1, 0, 1, 1, 0, 2, 1, 2, 2, 0, 0, 0, 2, 0, 1, 2, 0,
       0, 0, 1, 2, 2, 2, 2, 0, 1], dtype=int64)

In [29]:
y_real = np.argmax(testY, axis=1)

In [30]:
y_real

array([2, 0, 0, 0, 1, 2, 1, 1, 0, 0, 1, 2, 1, 1, 0, 1, 1, 0, 0, 1, 2, 2,
       0, 1, 0, 2, 2, 0, 1, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 2, 1, 1,
       1, 2, 2, 2, 1, 0, 1, 2, 1, 2, 2, 1, 0, 0, 1, 1, 0, 0, 1, 2, 0, 2,
       2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 0, 0, 1, 0, 1, 2, 0, 2, 2, 1, 0, 1,
       2, 2, 2, 1, 2, 1, 1, 1, 2, 0, 2, 2, 1, 2, 0, 1, 0, 2, 1, 2, 2, 1,
       0, 0, 2, 2, 2, 2, 2, 0, 2], dtype=int64)

In [31]:
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(y_real,y_pred)
# kappa = cohen_kappa_score(dataY[-30:],y_pred)
#(label除非是你想计算其中的分类子集的kappa系数，否则不需要设置)
kappa

0.5733417694822314

In [32]:
from sklearn.metrics import classification_report

In [33]:
print(classification_report(y_real,y_pred))

              precision    recall  f1-score   support

           0       0.69      0.91      0.78        34
           1       0.62      0.50      0.55        42
           2       0.82      0.77      0.80        43

    accuracy                           0.71       119
   macro avg       0.71      0.73      0.71       119
weighted avg       0.71      0.71      0.71       119



In [34]:
# model.save('fcn111.h5')

In [35]:
classification_report(y_real,y_pred,output_dict=True)

{'0': {'precision': 0.6888888888888889,
  'recall': 0.9117647058823529,
  'f1-score': 0.7848101265822784,
  'support': 34},
 '1': {'precision': 0.6176470588235294,
  'recall': 0.5,
  'f1-score': 0.5526315789473685,
  'support': 42},
 '2': {'precision': 0.825,
  'recall': 0.7674418604651163,
  'f1-score': 0.7951807228915662,
  'support': 43},
 'accuracy': 0.7142857142857143,
 'macro avg': {'precision': 0.710511982570806,
  'recall': 0.7264021887824897,
  'f1-score': 0.710874142807071,
  'support': 119},
 'weighted avg': {'precision': 0.7129277201076509,
  'recall': 0.7142857142857143,
  'f1-score': 0.7066121151590276,
  'support': 119}}